In [4]:
import sys
import os.path
import time
import re
import paramiko

In [5]:
#Asking user to provide the USERNAME/PASSWORD FILE
user_file = input("Provide the correct file path containing the USERNAME/PASSWORD !") #accepting the file path    #checking if the path is valid
    
if os.path.isfile(user_file) == True:
    print("USERNAME/PASSWORD FILE EXISTS")
else :
    print("File {} doesn't exist! Provide the correct path! ".format(user_file))
    sys.exit()

#Asking user the command file
command_file = input("Provide the correct file path containing the command !") #accepting the file path
    
    #checking if the path is valid
if os.path.isfile(command_file) == True:
    print("COMMAND FILE EXISTS")
else :
    print("File {} doesn't exist! Provide the correct path! ".format(command_file))
    sys.exit()

#Open the SSH connection for the arista switches
def ssh_connection(ip):
    global user_file
    global command_file
    
    try:
        the_user_file = open(user_file,'r')
    
        the_user_file.seek(0)
    
        username = the_user_file.readlines()[0].split(',')[1].rstrip('\n') #username extracted from the file
    
        the_user_file.seek(0)
    
        password = the_user_file.readlines()[0].split(',')[1].rstrip('\n') #password extracted from the file
        
        #Starting the SSH session
        session = paramiko.SSHClient()
        
        #Handling unknown session keys in testing environment 
        session.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        
        #Connecting to the session in remote and starting it 
        session.connect(ip.strip('\n'),username = username,password = password)
        
        #Starting the session
        start_session = session.invoke_shell()
        
        #Sending commands and disabling pagination
        start_session.send("\n")
        start_session.send("terminal length 0\n")
        time.sleep(0)
        
        #Commands for Config mode
        start_session.send("\n")
        start_session.send("configure terminal\n")
        time.sleep(1)
        
        #Opening the Command file by user
        the_cmd_file = open(command_file,'r')
    
        the_cmd_file.seek(0)
        
        #Executing each line in the command file
        for line in the_cmd_file:
            start_session.send(line + '\n')
            time.sleep(2)
        
        #Close all the files
        user_file.close()
        command_file.close()
        
        #Receving the outputs
        
        output = start_session.recv(65335)
        
        #Checking for errors
        if re.search(b"%Invalid input" ,output):
            print("One or more Syntax errors were found on {}".format(ip))
        else:
            print("All good for {}".format(ip))
        
        #Printing the received Outputs
        print(str(output) +'/n')
        
        #Close all session
        session.close()
    except paramiko.AuthenticationException:
        print("Invalid username or password \n Please check and enter again")
        print("Program closed ......bye")   

Provide the correct file path containing the USERNAME/PASSWORD !ok
File ok doesn't exist! Provide the correct path! 


SystemExit: 

c:\users\vishakadatta\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
